In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm

from HSDM_simulation import simulate_HSDM_2D # for simulating circular diffusion models
from HSDM_likelihood import likelihood_2D # for calculating the likelihood and model fitting
from scipy.optimize import differential_evolution # for minimizing the  negative log-likelihood function

In [2]:
# This function simulates experimental data
def simulate_experiment(n_subject=10, n_trials=250, 
                        min_threshold=0.5, max_threshold=5,
                        min_lambda=0.1, max_lambda=2,
                        min_ndt=0.1, max_ndt=1,
                        min_mu=-3, max_mu=3, 
                        collapsing=False):

    df = {'subject':[],
          'trial':[],
          'response':[],
          'rt':[]}

    for s in range(n_subject):
        threshold = np.random.uniform(min_threshold, max_threshold)
        lamb = np.random.uniform(min_lambda, max_lambda)
        if collapsing:
            a = lambda t: threshold - lamb*t
        else:
            a = lambda t: threshold

        ndt = np.random.uniform(min_ndt, max_ndt)
        mu = np.array([np.random.uniform(min_mu, max_mu), 
                       np.random.uniform(min_mu, max_mu)])

        for n in range(n_trials):
            rt, theta = simulate_HSDM_2D(a, mu, ndt)
            
            df['subject'].append(s+1)
            df['trial'].append(n+1)
            df['response'].append(theta)
            df['rt'].append(rt)
            
    return pd.DataFrame(df)


# Data simulation:

In the real modeling you can just load your own data and there is no need to simulate data!

In [3]:
data = simulate_experiment(n_subject=10, n_trials=300, collapsing=False)

In [4]:
data

,subject,trial,response,rt
0,1,1,0.625981,0.846181
1,1,2,0.666640,0.768181
2,1,3,0.679476,0.697181
3,1,4,2.830328,0.714181
4,1,5,3.053317,0.636181
...,...,...,...,...
2995,10,296,1.945552,1.114302
2996,10,297,1.725686,1.387302
2997,10,298,2.207019,1.474302
2998,10,299,2.390930,1.736302


# Model fitting:

In [5]:
min_threshold = 0.5
max_threshold = 5

min_lambda = .1
max_lambda = 2

min_ndt = 0.1
max_ndt = 1

min_mu = -3
max_mu = 3

In [6]:
subject_prms = {'subject': [],
                'threshold': [],
                'ndt': [],
                'drift_x': [],
                'drift_y': []}

for sbj in tqdm(data.subject.unique()):
    sbj_data = data[data['subject']==sbj]
    
    # minimizing the negative log-likelihood function for each participants
    min_ans = differential_evolution(likelihood_2D,
                                     args=(sbj_data['rt'].values, sbj_data['response'].values, False, 0.1),
                                     bounds=[(min_threshold, max_threshold), (min_ndt, max_ndt),
                                             (min_mu, max_mu), (min_mu, max_mu)])
    
    subject_prms['subject'].append(sbj)
    subject_prms['threshold'].append(min_ans.x[0])
    subject_prms['ndt'].append(min_ans.x[1])
    subject_prms['drift_x'].append(min_ans.x[2])
    subject_prms['drift_y'].append(min_ans.x[3])
    
subject_prms = pd.DataFrame(subject_prms)

100%|███████████████████████████████████████████| 10/10 [01:24<00:00,  8.46s/it]


In [7]:
subject_prms

,subject,threshold,ndt,drift_x,drift_y
0,1,0.718027,0.580777,0.597487,2.977751
1,2,1.400457,0.855432,-0.687114,0.066868
2,3,0.959124,0.262809,2.675127,-3.000000
3,4,4.443905,0.746515,-0.126802,-2.864357
4,5,1.641631,0.586198,1.089410,0.781404
5,6,0.847980,0.609040,2.581304,-2.238526
6,7,0.705684,0.449780,-2.568188,-0.113110
7,8,2.021401,0.891075,2.569735,-1.874579
8,9,2.093224,0.950822,-1.735835,-2.389622
9,10,3.030350,0.407024,0.232113,2.440001
